In [6]:
import json
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from getpass import getpass

def decrypt_bitwarden_data(encrypted_data, master_password):

    # Extract the encryption parameters
    salt = base64.b64decode(encrypted_data['salt'])
    
    iterations = encrypted_data['kdfIterations']
    
    # Generate the master key using PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
        backend=default_backend()
    )
    master_key = kdf.derive(master_password.encode())
    
    # Function to decrypt an encrypted string
    def decrypt_string(encrypted_str):
        if not encrypted_str.startswith('2.'):
            raise ValueError("Unsupported encryption version")
        
        encrypted_str = encrypted_str[2:]  # Remove version prefix
        parts = encrypted_str.split('|')
        if len(parts) != 2:
            raise ValueError("Invalid encrypted string format")
        
        iv = base64.b64decode(parts[0])
        ct = base64.b64decode(parts[1])
        
        cipher = Cipher(
            algorithms.AES(master_key),
            modes.CBC(iv),
            backend=default_backend()
        )
        
        decryptor = cipher.decryptor()
        padded_data = decryptor.update(ct) + decryptor.finalize()
        
        unpadder = padding.PKCS7(128).unpadder()
        return unpadder.update(padded_data) + unpadder.finalize()
    
    # Verify the master password using encKeyValidation
    try:
        validation_str = encrypted_data['encKeyValidation_DO_NOT_EDIT']
        decrypt_string(validation_str)
    except Exception:
        raise ValueError("Invalid master password")
    
    # Decrypt the main data
    decrypted_data = decrypt_string(encrypted_data['data'])
    return json.loads(decrypted_data)

def main():
    # Get the password from user
    password = getpass("Enter your master password: ")
    
    try:
        # Load the encrypted data
        encrypted_data = {
            "encrypted": True,
            "passwordProtected": True,
            "salt": input_data['salt'],
            "kdfType": input_data['kdfType'],
            "kdfIterations": input_data['kdfIterations'],
            "encKeyValidation_DO_NOT_EDIT": input_data['encKeyValidation_DO_NOT_EDIT'],
            "data": input_data['data']
        }
        
        # Decrypt the data
        decrypted_data = decrypt_bitwarden_data(encrypted_data, password)
        
        # Save to a file
        output_file = "decrypted_bitwarden_export.json"
        with open(output_file, 'w') as f:
            json.dump(decrypted_data, f, indent=2)
        
        print(f"Successfully decrypted! Saved to {output_file}")
        
    except ValueError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")

if __name__ == "__main__":
    main()

Error: Invalid master password


In [8]:
import json
import base64
import getpass
import sys
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def decode_base64(b64string):
    """Decode base64 to bytes, with padding handling"""
    # Add padding if needed
    padding = len(b64string) % 4
    if padding:
        b64string += '=' * (4 - padding)
    return base64.b64decode(b64string)

def derive_key(password, salt, iterations):
    """Derive encryption key using PBKDF2"""
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 256 bit key
        salt=salt,
        iterations=iterations,
        backend=default_backend()
    )
    return kdf.derive(password.encode('utf-8'))

def decrypt_data(encrypted_string, key):
    """Decrypt Bitwarden encrypted string format"""
    # Bitwarden format: [encryption_type].[iv]|[ciphertext]
    parts = encrypted_string.split('.')
    if len(parts) != 2:
        raise ValueError(f"Invalid encrypted string format: {encrypted_string}")
    
    enc_type = parts[0]
    if enc_type != "2":  # Type 2 is AES-CBC
        raise ValueError(f"Unsupported encryption type: {enc_type}")
    
    iv_ciphertext = parts[1].split('|')
    if len(iv_ciphertext) != 2:
        raise ValueError(f"Invalid IV/ciphertext format: {parts[1]}")
    
    iv = decode_base64(iv_ciphertext[0])
    ciphertext = decode_base64(iv_ciphertext[1])
    
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Remove PKCS#7 padding
    padding_length = padded_plaintext[-1]
    plaintext = padded_plaintext[:-padding_length]
    return plaintext.decode('utf-8')

def validate_key(master_key, validation_string):
    """Validate the master key using encKeyValidation"""
    try:
        # Extract parts from the validation string
        parts = validation_string.split('|')
        if len(parts) != 3:
            return False
        
        # Decrypt the validation data
        decrypted = decrypt_data(parts[0] + '.' + parts[1] + '|' + parts[2], master_key)
        return True  # If decryption succeeds without error
    except Exception:
        return False

def process_item(item, master_key):
    """Process and decrypt a single item from the vault"""
    decrypted_item = {}
    
    # Common fields to decrypt
    encrypted_fields = ['name', 'notes']
    
    # Copy non-encrypted fields
    for key, value in item.items():
        if key not in encrypted_fields and not isinstance(value, dict):
            decrypted_item[key] = value
    
    # Decrypt simple encrypted fields
    for field in encrypted_fields:
        if field in item and item[field]:
            try:
                decrypted_item[field] = decrypt_data(item[field], master_key)
            except Exception as e:
                decrypted_item[field] = f"[Decryption failed: {str(e)}]"
    
    # Handle login data specially
    if 'login' in item and item['login']:
        decrypted_item['login'] = {}
        login = item['login']
        
        # Copy non-encrypted fields
        for key, value in login.items():
            if key not in ['username', 'password', 'totp', 'uris']:
                decrypted_item['login'][key] = value
        
        # Decrypt login fields
        login_fields = ['username', 'password', 'totp']
        for field in login_fields:
            if field in login and login[field]:
                try:
                    decrypted_item['login'][field] = decrypt_data(login[field], master_key)
                except Exception as e:
                    decrypted_item['login'][field] = f"[Decryption failed: {str(e)}]"
        
        # Handle URIs
        if 'uris' in login and login['uris']:
            decrypted_item['login']['uris'] = []
            for uri in login['uris']:
                decrypted_uri = {}
                for key, value in uri.items():
                    if key == 'uri' and value:
                        try:
                            decrypted_uri[key] = decrypt_data(value, master_key)
                        except Exception as e:
                            decrypted_uri[key] = f"[Decryption failed: {str(e)}]"
                    else:
                        decrypted_uri[key] = value
                decrypted_item['login']['uris'].append(decrypted_uri)
    
    # Handle card data if present
    if 'card' in item and item['card']:
        decrypted_item['card'] = {}
        card = item['card']
        
        card_fields = ['cardholderName', 'brand', 'number', 'expMonth', 'expYear', 'code']
        for field in card_fields:
            if field in card and card[field]:
                try:
                    decrypted_item['card'][field] = decrypt_data(card[field], master_key)
                except Exception as e:
                    decrypted_item['card'][field] = f"[Decryption failed: {str(e)}]"
    
    # Handle identity data if present
    if 'identity' in item and item['identity']:
        decrypted_item['identity'] = {}
        identity = item['identity']
        
        identity_fields = ['title', 'firstName', 'middleName', 'lastName', 'address1', 
                          'address2', 'address3', 'city', 'state', 'postalCode', 
                          'country', 'company', 'email', 'phone', 'ssn', 'username', 
                          'passportNumber', 'licenseNumber']
        
        for field in identity_fields:
            if field in identity and identity[field]:
                try:
                    decrypted_item['identity'][field] = decrypt_data(identity[field], master_key)
                except Exception as e:
                    decrypted_item['identity'][field] = f"[Decryption failed: {str(e)}]"
    
    # Handle secure notes (just containing encrypted notes field)
    # Already handled in the common fields
    
    return decrypted_item

def main():

    file_path = "icecream.json"
    
    # Get master password
    master_password = getpass.getpass("Enter your Bitwarden master password: ")
    
    try:
        # Load the encrypted file
        with open(file_path, 'r') as f:
            encrypted_data = json.load(f)
        
        # Extract encryption parameters
        salt = decode_base64(encrypted_data["salt"])
        iterations = encrypted_data["kdfIterations"]
        
        # Derive the master key
        master_key = derive_key(master_password, salt, iterations)
        
        # Validate the master key using encKeyValidation
        if "encKeyValidation_DO_NOT_EDIT" in encrypted_data:
            validation = encrypted_data["encKeyValidation_DO_NOT_EDIT"]
            if not validate_key(master_key, validation):
                print("Invalid master password or corrupted data.")
                sys.exit(1)
            print("Master password validated successfully.")
        
        # Decrypt items in the data field
        if "data" in encrypted_data and isinstance(encrypted_data["data"], list):
            print(f"Found {len(encrypted_data['data'])} items to decrypt...")
            
            decrypted_items = []
            for item in encrypted_data["data"]:
                decrypted_item = process_item(item, master_key)
                decrypted_items.append(decrypted_item)
            
            # Write decrypted data to file
            output_file = 'decrypted_bitwarden.json'
            with open(output_file, 'w') as f:
                json.dump(decrypted_items, f, indent=2)
            
            print(f"Decryption completed. Data saved to '{output_file}'")
        else:
            print("No data field found or unexpected format.")
    
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

if __name__ == "__main__":
    main()

Invalid master password or corrupted data.


SystemExit: 1

/Users/furqaan/Desktop/Projects/AI-Practice/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
